# HULAT-UC3M CANTEMIST Participation

Team Participants:

* Sergio Santamaria Carrasco
* Paloma Mártinez Fernández

## Settings

1. Set the parent dictory
2. Check GPU

In [1]:
sst_home='/home/sergio/Descargas/TFM/TFM/' 

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Text Format

In [2]:
file_text = sst_home + 'train-set-to-publish/cantemist-ner/cc_onco1.txt'

with open(file_text, 'r') as file:
  print(file.read())

ANAMNESIS
Mujer de 67 años con antecedentes personales de hipotiroidismo en tratamiento con levotiroxina y fumadora activa de 12.5 paquetes/año. Consulta en Urgencias por sensación progresiva de “acorchamiento y hormigueos” en ambos miembros superiores e inferiores, así como una dificultad progresiva para la deambulación de 4 meses de evolución, asociando asimismo alteración de la memoria reciente desde el último mes

EXPLORACIÓN FÍSICA
Presentaba una exploración cardiopulmonar y abdominal normal. En la exploración neurológica destaca balance motor por grupos musculares conservado; arreflexia rotuliana y aquilea e hiporreflexia bicipital; tetrahipoestesia asimétrica (táctil, algésica, vibratoria y posicional) en patrón de “guante y calcetín altos” de predominio izquierdo; Romberg positivo y marcha con leve aumento de base de sustentación que impresiona ataxia sensitiva.

PRUEBAS COMPLEMENTARIAS
En las exploraciones complementarias, hemograma, bioquímica y coagulación no presentaban alt

## Annotations Format

In [3]:
file_ann = sst_home + 'train-set-to-publish/cantemist-ner/cc_onco1.ann'

with open(file_ann) as file:
  print(file.read())

T1	MORFOLOGIA_NEOPLASIA 2719 2740	Carcinoma microcítico
T2	MORFOLOGIA_NEOPLASIA 2950 2971	carcinoma microcítico
T3	MORFOLOGIA_NEOPLASIA 2988 2990	M0



## Requirements

1. Tensorflow 1.14.0
2. Keras 2.2.4
3. Fasttext 0.2.0
4. MeaningCloud-python
5. Spacy
6. Sklearn_crfsuite
7. Keras-contrib
8. Pandas

In [ ]:
!pip install MeaningCloud-python

In [ ]:
!python -m spacy download es_core_news_md

import spacy
import es_core_news_md

nlp = es_core_news_md.load()

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

### Syllabizer

Using the rules of Spanish orthography, the syllabizer allows a word to be broken into its component syllables. The necessary code has been extracted from https://github.com/mabodo/sibilizador/blob/master/Silabizator.ipynb

In [ ]:
class char():
    def __init__(self):
        pass
    
class char_line():
    def __init__(self, word):
        self.word = word
        self.char_line = [(char, self.char_type(char)) for char in word]
        self.type_line = ''.join(chartype for char, chartype in self.char_line)
        
    def char_type(self, char):
        if char in set(['a', 'á', 'e', 'é','o', 'ó', 'í', 'ú']):
            return 'V' #strong vowel
        if char in set(['i', 'u', 'ü']):
            return 'v' #week vowel
        if char=='x':
            return 'x'
        if char=='s':
            return 's'
        else:
            return 'c'
            
    def find(self, finder):
        return self.type_line.find(finder)
        
    def split(self, pos, where):
        return char_line(self.word[0:pos+where]), char_line(self.word[pos+where:])
    
    def split_by(self, finder, where):
        split_point = self.find(finder)
        if split_point!=-1:
            chl1, chl2 = self.split(split_point, where)
            return chl1, chl2
        return self, False
     
    def __str__(self):
        return self.word
    
    def __repr__(self):
        return repr(self.word)

class silabizer():
    def __init__(self):
        self.grammar = []
        
    def split(self, chars):
        rules  = [('VV',1), ('cccc',2), ('xcc',1), ('ccx',2), ('csc',2), ('xc',1), ('cc',1), ('vcc',2), ('Vcc',2), ('sc',1), ('cs',1),('Vc',1), ('vc',1), ('Vs',1), ('vs',1)]
        for split_rule, where in rules:
            first, second = chars.split_by(split_rule,where)
            if second:
                if first.type_line in set(['c','s','x','cs']) or second.type_line in set(['c','s','x','cs']):
                    #print 'skip1', first.word, second.word, split_rule, chars.type_line
                    continue
                if first.type_line[-1]=='c' and second.word[0] in set(['l','r']):
                    continue
                if first.word[-1]=='l' and second.word[-1]=='l':
                    continue
                if first.word[-1]=='r' and second.word[-1]=='r':
                    continue
                if first.word[-1]=='c' and second.word[-1]=='h':
                    continue
                return self.split(first)+self.split(second)
        return [chars]
        
    def __call__(self, word):
        return self.split(char_line(word))

### Getting Meaning Cloud Entities

MeaningCloud is a Software as a Service product that allows users to embed text analysis and semantic processing into any application or system. Its API, Topic's Extraction, allows extracting the entities present in a text, such as drugs or places. In addition, using a customized dictionary built from the different terms present in the National Cancer Institute's dictionary, we also identify cancer treatments, types of cancer, medical tests, etc.

The code above let us extract the entities present in the medical texts and store this information in a dataframe.
To be used, is needed has a license key. You can find more information in https://www.meaningcloud.com/

In [ ]:
import meaningcloud
import os

def getMeaningCloudEntities(license_key, sst_home_path):
    data = []
    for file in [file[:-4] for file in os.listdir(sst_home_path) if file.endswith('.txt') and not ' ' in file]:
        file_text = os.path.join(sst_home_path, file + '.txt')

        with open(file_text) as f:
            text = f.read()

        try:
            # We are going to make a request to the Topics Extraction API
            otherParameters = {'rt': 'y', 'ud': 'Codiesp_CIE10_Diagnostics|cancer_INC'}
            topics_response = meaningcloud.TopicsResponse(meaningcloud.TopicsRequest(license_key, txt=text, lang='es',
                                                                                     topicType='ec', otherparams = otherParameters).sendReq())

            # If there are no errors in the request, we print the output
            if topics_response.isSuccessful():
                #print("\nThe request to 'Topics Extraction' finished successfully!\n")

                concepts = topics_response.getConcepts()
                if concepts:
                    #print("\tConcepts detected (" + str(len(entities)) + "):\n")
                    for concept in concepts:
                        form = topics_response.getTopicForm(concept)
                        form_type = topics_response.getTypeLastNode(topics_response.getOntoType(concept))
                        is_user = topics_response.isUserDefined(concept)
                        for appear in concept['variant_list']:
                            data.append({'DOC_ID': file, 'FORMA': form, 'TIPO': 'Concept', 'CLASE': form_type, 'INICIO': appear['inip'], 'FINAL': appear['endp'], 'IS_USER_DEFINED': is_user})
                        #print("\t\t" + topics_response.getTopicForm(concept) + ' --> ' +
                              #topics_response.getTypeLastNode(topics_response.getOntoType(concept)) + "\n")

                entities = topics_response.getEntities()        
                if entities:
                    #print("\tEntities detected (" + str(len(entities)) + "):\n")
                    for entity in entities:
                        form = topics_response.getTopicForm(entity)
                        form_type = topics_response.getTypeLastNode(topics_response.getOntoType(entity))
                        is_user = topics_response.isUserDefined(concept)
                        for appear in entity['variant_list']:
                            data.append({'DOC_ID': file, 'FORMA': form, 'TIPO': 'Entity', 'CLASE': form_type, 'INICIO': appear['inip'], 'FINAL': appear['endp'], 'IS_USER_DEFINED': is_user})
                        #print("\t\t" + topics_response.getTopicForm(entity) + ' --> ' +
                              #topics_response.getTypeLastNode(topics_response.getOntoType(entity)) + "\n")

                else:
                    print("\tNo entities detected!\n")
            else:
                if topics_response.getResponse() is None:
                    print("\nOh no! The request sent did not return a Json\n")
                else:
                    print("\nOh no! There was the following error: " + topics_response.getStatusMsg() + "\n")

        except ValueError:
            e = sys.exc_info()[0]
            print("\nException: " + str(e))

    df = pd.DataFrame(data)
    return df

In [ ]:
def getMeaningCloudDict(df, doc_id):
    data = {}
    df = df[df['DOC_ID'] == doc_id]
    for instance in df[['FORMA', 'CLASE', 'INICIO', 'FINAL']].iterrows():
        row = instance[1]
        tokens = nlp(row['FORMA'])
        for token in tokens:
            ini = row['INICIO'] + token.idx
            final = ini + len(token)
            data[(ini, final)] = data.get((ini, final), []) + [row['CLASE']]

    return data

### BRAT to BIOES

The next code transform the BRAT annotations into BIOES format. The code return a dict with the token offset and the BIOES schema. In this schema,tokens are annotated using the following tags:

* B: represents a token that conform the begining of an entity.
* I: indicate that the token belongs to an entity.
* O: represents that the token does not belong to an entity.
* E: marks a token as the end of a given entity.
* S: indicates that an entity is comprised of a single token.

In [ ]:
############# BIOES NOTATION #####################
BEGIN = 'B'
INSIDE = 'I'
OUTSIDE = 'O'
END = 'E'
SINGLE = 'S'

def getDictEntities(file_ann):
  entities = {}
  with open(file_ann) as anns:
    for ann in anns:
      ent = ann[:-1].split('\t')[2]
      ent = nlp(ent)
      start = int(ann[:-1].split('\t')[1].split(' ')[1])
      end = int(ann[:-1].split('\t')[1].split(' ')[2])
      if (len(ent) == 1):
        entities[(start, end)] = SINGLE + '_' + ann.split('\t')[1].split(' ')[0]
      else:
        entities[(start, start + len(ent[0].text))] = BEGIN + '_' + ann.split('\t')[1].split(' ')[0]
        entities[(end - len(ent[-1].text)), end] = END + '_' + ann.split('\t')[1].split(' ')[0]
        for i in range(len(ent) - 2):
          spaces = (ent[i + 1].idx) - (ent[i].idx + len(ent[i].text))
          start = start + len(ent[i].text) + spaces
          entities[(start, start + len(ent[i + 1].text))] = INSIDE + '_' + ann.split('\t')[1].split(' ')[0]

  return entities

# Deep Learning System

## Preparing data for model input

We pre-process the text of the clinical cases taking into account different steps.  First, thetexts are split into tokens and sentences using the Spacy, an open-source library that providessupport for texts in several languages, including Spanish.

We generate the different attributes considered to be the input of the deep learning stack

1. Words: A representation based on pre-trained word embeddings has been used. These vectors have a total of 300 dimensions.

2. Part-of-speech: This feature has been considered due to the significant amount ofinformation it offers about the word and its neighbors. It can also help in word sensedisambiguation. The PoS-Tagging model used was the one provided by the Spacy. Anembedding representaton of this feature is learned during training, resulting in a 40-dimensional vector (20 PoS + 20 TAG).

3. Characters: We also add character-level embeddings of the words, learned during train-ing and resulting in a 30-dimensional vector. These have proven to be useful for specific-domain tasks and morphologically-rich languages.

4. Syllables: Syllable-level embeddings of the words, learned during training and resultingin a 75-dimensional vector is also added. Like character-level embeddings, they help todeal with words outside the vocabulary and contribute to capturing common prefixesand suffixes in the domain and correctly classifying words

5. Meaning Cloud Topic Extaction


In [ ]:
import os
import re
from sklearn.preprocessing import MultiLabelBinarizer

def getElements(sst_home, max_len_seq, concepts_df, classes, getTags = True):
  _words = dict()
  _doc_tags = {}
  _entities = {}
  _docs = {}
  _docs_offset = {}
  mlb = MultiLabelBinarizer(classes = classes)
    
  for file in [file[:-4] for file in os.listdir(sst_home) if file.endswith('.txt') and not ' ' in file]:
    concepts_dict = getMeaningCloudDict(concepts_df, file)
    
    file_text = os.path.join(sst_home, file + '.txt')
    if getTags:
      file_ann = os.path.join(sst_home, file + '.ann')
      _entities = getDictEntities(file_ann)
    with open(file_text) as f:
      #re_date = re.compile(r"\d{1,2}/\d{1,2}/\d{4}")
      _sents = []
      _sents_lemma = []
      _tags = []
      _sents_pos = []
      _sents_gtags = []
      _sents_offset = []
      _sents_concepts = []
      text = f.read()
      sections = text.split('\n\n')
      offset = 0
      for section in sections:
        sents = section.split('\n')
        for sent in sents:
          sent_nlp = nlp(sent)
          stop_sent = [token for token in sent_nlp if not token.is_stop]
          for i in range(0, len(stop_sent), max_len_seq):
            _sent = []
            _sent_tags = []
            _sent_pos = []
            _sent_gtags = []
            _sent_offset = []
            _sent_concepts = []
            _sent_lemma = []
            for token in stop_sent[i:i+max_len_seq]:
              _sent.append(token.text)
              _sent_lemma.append(token.lemma_)
              _sent_offset.append((token.text, token.idx + offset, token.idx + offset + len(token.text)))
              _entity = _entities.get((token.idx + offset, token.idx + offset + len(token.text)), OUTSIDE)
              _sent_tags.append(_entity)
              _words[token.text] = _words.get(token.text, 0) + 1
              _sent_pos.append(token.pos_)
              _sent_gtags.append(token.tag_.split('_')[0])
              _sent_concepts.append(concepts_dict.get((token.idx + offset, token.idx + offset + len(token.text)), []))

            _sents.append(_sent)
            _sents_lemma.append(_sent_lemma)
            _tags.append(_sent_tags)
            _sents_pos.append(_sent_pos)
            _sents_gtags.append(_sent_gtags)
            _sents_offset.append(_sent_offset)
            _mlb_concepts = mlb.fit_transform(_sent_concepts)
            _sents_concepts.append(_mlb_concepts)
            

          offset = offset + len(sent) + 1
        offset = offset + 1

      _docs[file] = (_sents, _sents_pos, _sents_gtags, _sents_concepts, _sents_lemma)
      _doc_tags[file] = _tags
      _docs_offset[file] = _sents_offset

  #_words = list(_words)

  return _words, _docs, _doc_tags, _docs_offset

In [ ]:
def get2idx(words, chars, pos, gramm_tags, sylls):
  TM = '_MORFOLOGIA_NEOPLASIA'
  word2idx = {w: i + 2 for i, w in enumerate(words)}
  word2idx["#ENDPAD"] = 0
  word2idx["UNK"] = 1

  char2idx = {char:i + 2 for i,char in enumerate(chars)}
  char2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  char2idx["UNK"] = 1

  pos2idx = {pos:i + 2 for i,pos in enumerate(pos)}
  pos2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  pos2idx["UNK"] = 1

  grammtags2idx = {tag:i + 2 for i,tag in enumerate(gramm_tags)}
  grammtags2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  grammtags2idx["UNK"] = 1

  sylls2idx = {syll:i + 2 for i,syll in enumerate(sylls)}
  sylls2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  sylls2idx["UNK"] = 1

  tag2idx = {tag:i + 1 for i,tag in enumerate([BEGIN + TM, INSIDE + TM, END + TM, OUTSIDE, SINGLE + TM])}
  tag2idx["#ENDPAD"] = 0 

  return word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx

In [ ]:
def getCharacterInput(sentences, max_seq_len, max_chars_len, char2idx):
  X_char = []
  for sentence in sentences:
    sent_seq = []
    # max_len = 75
    for i in range(max_seq_len):
      word_seq = []
      # char sequence for words
      for j in range(max_chars_len):
        try:
          # chars of specific sentence of i
          word_seq.append(char2idx.get(sentence[i][j], 1)) 
        except:  # if char-sequence is out of range , pad it with "PAD" tag
          word_seq.append(char2idx.get("#ENDPAD"))

      sent_seq.append(word_seq)
    # append sentence sequences as character-by-character to X_char for Model input
    X_char.append(np.array(sent_seq))

  return np.array(X_char)

def getSyllsInput(sentences, max_seq_len, max_sylls_len, sylls2idx):
  X_syll = []
  for sentence in sentences:
    sent_seq = []
    # max_len = 75
    for i in range(max_seq_len):
      word_seq = []
      syllables = []
      try:
        syllables = silabizer(sentence[i])
      except:
        pass
      for j in range(max_sylls_len):
        try:
          # chars of specific sentence of i
          word_seq.append(sylls2idx.get(str(syllables[j]).lower(), 1)) 
        except:  # if char-sequence is out of range , pad it with "PAD" tag
          word_seq.append(sylls2idx.get("#ENDPAD"))

      sent_seq.append(word_seq)
    # append sentence sequences as character-by-character to X_char for Model input
    X_syll.append(np.array(sent_seq))

  return np.array(X_syll)

from keras.preprocessing.sequence import pad_sequences

def getWordInput(sentences, max_seq_len, word2idx):
  X = [[word2idx.get(w,1) for w in s[:max_seq_len]] for s in sentences]
  X = pad_sequences(maxlen = max_seq_len, sequences = X, truncating= 'post', padding ='post', value=0 )
  return np.array(X)

def getPosInput(sentences, max_seq_len, pos2idx):
  X_pos = [[pos2idx.get(w,1) for w in s] for s in sentences]
  X_pos = pad_sequences(maxlen = max_seq_len, sequences = X_pos, truncating= 'post', padding ='post', value=0 )
  return np.array(X_pos)

def getGTagInput(sentences, max_seq_len, grammtags2idx):
  X_gramm_tags = [[grammtags2idx.get(w,1) for w in s] for s in sentences]
  X_gramm_tags = pad_sequences(maxlen = max_seq_len, sequences = X_gramm_tags, truncating= 'post', padding ='post', value=0 )
  return np.array(X_gramm_tags)

def getConceptsInput(sentences, max_seq_len, classes):
  X = pad_sequences(maxlen = max_seq_len, sequences = sentences, truncating= 'post', padding ='post', value=np.zeros(len(classes)) )
  return np.array(X)

from keras.utils.np_utils import to_categorical

def getY(tags, max_seq_len, tag2idx):
  y = [[tag2idx[t] for t in sent_tags] for sent_tags in tags]
  y = pad_sequences(maxlen = max_seq_len, sequences = y, padding = "post", truncating = "post", value = tag2idx["#ENDPAD"])
  
  y = [to_categorical(i, num_classes = 6) for i in y]
  return np.array(y)

### Word embeddings from Spanish Medical Corpora

The word embeddings used can be found in https://www.aclweb.org/anthology/W19-1916/

In [ ]:
import fasttext
import fasttext.util

sst_home_embeddings = sst_home + 'fast-text-model/'
ft = fasttext.load_model(sst_home_embeddings + 'cantemist-resource.bin')

def getEmbeddingMatrix(words2idx, emb_dim, model):
  embedding_matrix = np.zeros((len(words2idx), emb_dim))
  for word, i in words2idx.items():
    embedding_matrix[i] = model[word]

  return embedding_matrix

Cause we are using pre-trained embeddings, we use as our vocabulary the words contained in both the training, development and test sets

In [ ]:
_total_words = set()
sst_home_test = sst_home + 'train-set-to-publish/cantemist-ner/'
for file in [file[:-4] for file in os.listdir(sst_home_test) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home_test, file + '.txt')
    
    with open(file_text) as f:
        text = f.read()
        doc = nlp(text)
        for token in doc:
            if not token.is_stop:
                _total_words.add(token.text)
             
sst_home_test = sst_home + 'dev-set-to-publish/'
for file in [file[:-4] for file in os.listdir(sst_home_test) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home_test, file + '.txt')
    
    with open(file_text) as f:
        text = f.read()
        doc = nlp(text)
        for token in doc:
            if not token.is_stop:
                _total_words.add(token.text)
                
sst_home_test = sst_home + 'test-background-set-to-publish/'
for file in [file[:-4] for file in os.listdir(sst_home_test) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home_test, file + '.txt')
    
    with open(file_text) as f:
        text = f.read()
        doc = nlp(text)
        for token in doc:
            if not token.is_stop:
                _total_words.add(token.text)
                
_words = list(_total_words)

### Generating Training Data

The next code generate the training data. We set the next hyperparameters:

1. Max sequence length in words: 50
2. Max sequence length in characters: 30
3. Max sequence length in syllables: 10
4. The entities to take into account during the extraction of Meaninging Cloud

In [ ]:
import numpy as np

sst_home_train = sst_home + 'final-training/'

MAX_CHARS_LEN = 30
MAX_SEQ_LEN = 50
MAX_SYLLS_LEN = 10

silabizer = silabizer()
classes = ['Top', 'Disease', 'DIAGNÓSTICOS', 'MedicalTest', 'BodyPart',
       'Cancer', 'CancerDetection', 'CancerTreatment', 'Drug', 'PhysicalExtentUnit', 'WeightUnit', 'SpaceUnit',
          'VolumeUnit','BenignTumor', 'Unit']

df_concepts = pd.read_csv(sst_home + 'update_info.csv')
df_concepts = df_concepts[df_concepts['CLASE'].isin(classes)]
df_concepts.drop_duplicates()

_words_in_tr, _docs, _docs_tags, _ = getElements(sst_home_train, MAX_SEQ_LEN, df_concepts, classes)
_words_in_tr = list(_words_in_tr.keys())
_chars = list(set(''.join(_words_in_tr)))
_sylls = list(set(str(syll).lower() for word in _words_in_tr for syll in silabizer(word)))


_sents_train = [sent for sents in _docs.values() for sent in sents[0]]
_sents_pos = [sent for sents in _docs.values() for sent in sents[1]]
_sents_gtags = [sent for sents in _docs.values() for sent in sents[2]]
_sents_concepts = [sent for sents in _docs.values() for sent in sents[3]]
_sents_lemma_train = [sent for sents in _docs.values() for sent in sents[4]]

_pos = set(pos for sent in _sents_pos for pos in sent)
_gramm_tags = set(gtag for sent in _sents_gtags for gtag in sent)

_tags = [tag for tags in _docs_tags.values() for tag in tags]

word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx = get2idx(_words, _chars, _pos, _gramm_tags, _sylls)

X_char_train = getCharacterInput(_sents_train, MAX_SEQ_LEN, MAX_CHARS_LEN, char2idx)
X_words_train = getWordInput(_sents_train, MAX_SEQ_LEN, word2idx)
X_pos_train = getPosInput(_sents_pos, MAX_SEQ_LEN, pos2idx)
X_tag_train = getGTagInput(_sents_gtags, MAX_SEQ_LEN, grammtags2idx)
X_syll_train = getSyllsInput(_sents_train, MAX_SEQ_LEN, MAX_SYLLS_LEN, sylls2idx)
X_concepts_train = getConceptsInput(_sents_concepts, MAX_SEQ_LEN, classes)

y_train = getY(_tags, MAX_SEQ_LEN, tag2idx)

### Generating Validation Data

The next code generate the validation data.

In [ ]:
import numpy as np

sst_home_test = sst_home + 'dev-set1-to-publish/cantemist-ner/'

_, _docs, _docs_tags, offset = getElements(sst_home_test, MAX_SEQ_LEN, df_concepts, classes)

_sents_val = [sent for sents in _docs.values() for sent in sents[0]]
_sents_pos = [sent for sents in _docs.values() for sent in sents[1]]
_sents_gtags = [sent for sents in _docs.values() for sent in sents[2]]
_sents_concepts = [sent for sents in _docs.values() for sent in sents[3]]
_sents_lemma_test = [sent for sents in _docs.values() for sent in sents[4]]

_tags = [tag for tags in _docs_tags.values() for tag in tags]


X_char_test = getCharacterInput(_sents_val, MAX_SEQ_LEN, MAX_CHARS_LEN, char2idx)
X_words_test = getWordInput(_sents_val, MAX_SEQ_LEN, word2idx)
X_pos_test = getPosInput(_sents_pos, MAX_SEQ_LEN, pos2idx)
X_tag_test = getGTagInput(_sents_gtags, MAX_SEQ_LEN, grammtags2idx)
X_syll_test = getSyllsInput(_sents_val, MAX_SEQ_LEN, MAX_SYLLS_LEN, sylls2idx)
X_concepts_test = getConceptsInput(_sents_concepts, MAX_SEQ_LEN, classes)

y_test = getY(_tags, MAX_SEQ_LEN, tag2idx)


### Embedding Matrix 

The next code generate the embedding matrix to be used in the deep learning model

In [ ]:
embedding_matrix = getEmbeddingMatrix(word2idx, ft.get_dimension(), ft)
del ft

### Saving Data

We save the generated data so that it is not necessary to generate it again next time.

In [ ]:
import pickle

train_data = [X_char_train, X_words_train, X_pos_train, X_tag_train, X_syll_train, X_concepts_train, y_train]
with open(sst_home + 'data_final/train.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

test_data = [X_char_test, X_words_test, X_pos_test, X_tag_test, X_syll_test, X_concepts_test, y_test]
with open(sst_home + 'data_final/test.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(sst_home + 'data_final/we.pickle', 'wb') as handle:
    pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

two2idx = [word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx]
with open(sst_home + 'data_final/2idx.pickle', 'wb') as handle:
    pickle.dump(two2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Loading Data

In case the data is previously saved, we load these.

In [ ]:
import pickle

X_train = []
with open(sst_home + 'data_final/train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)

X_test = []
with open(sst_home + 'data_final/test.pickle', 'rb') as handle:
    X_test = pickle.load(handle)

embedding_matrixes = []
with open(sst_home + 'data_final/we.pickle', 'rb') as handle:
    embedding_matrix = pickle.load(handle)

two2idx = []
with open(sst_home + 'data_final/2idx.pickle', 'rb') as handle:
    two2idx = pickle.load(handle)

X_char_train, X_words_train, X_pos_train, X_tag_train, X_syll_train, X_concepts_train, y_train = X_train
X_char_test, X_words_test, X_pos_test, X_tag_test, X_syll_test, X_concepts_test, y_test = X_test
word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx = two2idx

## Deep Learning Architecture

The model implemented, as shown in the figure, works on two levels of maximum sequence word length.

First, a bidirectional LSTM layer receives the input features, the sequence of characters and syllables being previously processed by a convolutional and global max pooling block, for a maximum sequence length of 10. 

The reshaped output of this layer is concatenated with the input features, but this time for a maximum sequence length size of 50, and serves as the input for a new BiLSTM. This layer connects directly to a fully connected dense layer with 𝑡𝑎𝑛ℎ activation function.

The last layer (CRF optimization layer) consists of a conditional random fields layer. This layer receives as input the sequence of probabilities of the previous layer in order to improve predictions. This is due to the ability of the layer to take into account the dependencies between the different labels. The output of this layer provides the most probable sequence of labels.

![Architecture of the proposed model for named entity recognition.](imgs/model.png)

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Concatenate, Input, SpatialDropout1D
from keras.layers import Conv1D, MaxPooling1D,Flatten,GlobalMaxPooling1D, Reshape, RepeatVector
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint

CHAR_EMBEDDINGS_SIZE = 30    # Characters Embeddings Size
SYLL_EMBEDDINGS_SIZE = 75    # Syllable Embeddings Size
WORD_EMBEDDINGS_SIZE = 300   # Word Embeddings Size
POS_EMBEDDING_SIZE = 20      # PoS Embedding Size
GTAGS_EMBEDDING_SIZE = 20    # Tag Embedding Size

N_CLASSES = len(classes)     # Number of different entities in Meaning Cloud

MAX_CHARS_LEN = 30           # Max sequence char length
MAX_SEQ_LEN = 50             # Max sequence word length Level II
MAX_SYLLS_LEN = 10           # Max sequence sylls length

LOCAL_CONT_LEN = 10          # Max sequence word length Level I   
LOCAL_CONT_SPLIT = MAX_SEQ_LEN // LOCAL_CONT_LEN

CONV_FILTERS = 50            # Convolutional Filters in Character and Syllable Convolutional Layer
LSTM_UNITS = 300             # LSTM Units in both LSTM layers
DENSE_UNITS = 200            # Number of units in Dense layer  

####################### INPUT DATA ################################
word_input = Input(shape=(MAX_SEQ_LEN,), name = 'word_input')
pos_input = Input(shape=(MAX_SEQ_LEN,), name = 'pos_input')
gtag_input = Input(shape=(MAX_SEQ_LEN,), name = 'tag_input')
concepts_input = Input(shape=(MAX_SEQ_LEN, N_CLASSES), name = 'meaning_cloud_input')
char_input = Input(shape=(MAX_SEQ_LEN, MAX_CHARS_LEN), name = 'char_input')
sylls_input = Input(shape=(MAX_SEQ_LEN, MAX_SYLLS_LEN), name = 'sylls_input')


local_context_input = Reshape((LOCAL_CONT_SPLIT, LOCAL_CONT_LEN,), input_shape=(MAX_SEQ_LEN,))(word_input)

###################### CHARACTER SEQUENCE PROCCESSED BY CONVOLUTIONAL LAYER ################################
char_embedding = TimeDistributed(Embedding(input_dim = len(char2idx), output_dim = CHAR_EMBEDDINGS_SIZE, input_length=MAX_CHARS_LEN, name = 'char_embeddings', trainable = True))(char_input)
conv_1d = TimeDistributed(Conv1D(filters = CONV_FILTERS, kernel_size = 3,padding="valid", activation="relu", name="Conv1D_char"))(char_embedding)
conv_1d = TimeDistributed(Dropout(0.4))(conv_1d)
maxpool1d = TimeDistributed(GlobalMaxPooling1D(), name = 'max_pooling')(conv_1d)
char_enc = TimeDistributed(Flatten(), name = 'char_enc')(maxpool1d)

###################### SYLLABLE SEQUENCE PROCCESSED BY CONVOLUTIONAL LAYER ################################
syll_embedding = TimeDistributed(Embedding(input_dim = len(sylls2idx), output_dim = SYLL_EMBEDDINGS_SIZE, input_length=MAX_SYLLS_LEN, name = 'sylls_embeddings', trainable = True))(sylls_input)
conv_1d_syll = TimeDistributed(Conv1D(filters = CONV_FILTERS, kernel_size = 3,padding="valid", activation="relu", name="Conv1D_syll"))(syll_embedding)
conv_1d_syll = TimeDistributed(Dropout(0.4))(conv_1d_syll)
maxpool1d_syll = TimeDistributed(GlobalMaxPooling1D())(conv_1d_syll)
syll_enc = TimeDistributed(Flatten())(maxpool1d_syll)

##################### PoS + TAG EMBEDDINGS ################################################################
pos_embedding = Embedding(input_dim = len(pos2idx), output_dim = POS_EMBEDDING_SIZE, input_length = MAX_SEQ_LEN, name = 'pos_embeddings', trainable = True)(pos_input)
gtags_embedding = Embedding(input_dim = len(grammtags2idx), output_dim = GTAGS_EMBEDDING_SIZE, input_length = MAX_SEQ_LEN, name = 'gtags_embeddings', trainable = True)(gtag_input)

##################### WORD EMBEDDING LAYER ################################################################
embedding_layer = Embedding(input_dim = len(word2idx), output_dim = WORD_EMBEDDINGS_SIZE, weights=[embedding_matrix], trainable=True, name = 'word_embeddings')
word_embedding = embedding_layer(word_input)


#################### RESHAPE TO SET MAX SEQUENCE WORD LENGTH TO 10 ########################################
local_word_embedding = TimeDistributed(embedding_layer, name = 'local_context_embedding')(local_context_input)
local_char = Reshape((LOCAL_CONT_SPLIT, LOCAL_CONT_LEN, CONV_FILTERS), input_shape=(MAX_SEQ_LEN, CONV_FILTERS))(char_enc)
local_syll = Reshape((LOCAL_CONT_SPLIT, LOCAL_CONT_LEN, CONV_FILTERS), input_shape=(MAX_SEQ_LEN, CONV_FILTERS))(syll_enc)
local_pos = Reshape((LOCAL_CONT_SPLIT, LOCAL_CONT_LEN, POS_EMBEDDING_SIZE), input_shape=(MAX_SEQ_LEN, 100))(pos_embedding)
local_gtag = Reshape((LOCAL_CONT_SPLIT, LOCAL_CONT_LEN, GTAGS_EMBEDDING_SIZE), input_shape=(MAX_SEQ_LEN, 100))(gtags_embedding)
local_concepts = Reshape((LOCAL_CONT_SPLIT, LOCAL_CONT_LEN, N_CLASSES), input_shape=(MAX_SEQ_LEN, 100))(concepts_input)

#################### BiLSTM MAX SEQUENCE WORD LENGTH 10 ###################################################
local_info = Concatenate(axis = -1)([local_word_embedding, local_pos, local_gtag, local_char, local_syll, local_concepts])
local_cont = TimeDistributed(Bidirectional(LSTM(units=LSTM_UNITS, return_sequences=True, recurrent_dropout=0.4)))(local_info)
local_cont = TimeDistributed(Dropout(0.4))(local_cont)


################### RESHAPE TO SET MAX SEQUENCE WORD LENGTH TO 50 ##########################################
local_cont = Reshape((MAX_SEQ_LEN, 2*LSTM_UNITS), input_shape=(LOCAL_CONT_SPLIT, LOCAL_CONT_LEN, 2*LSTM_UNITS))(local_cont)

################### CONCATENATE BiLSTM OUTPUT AND INPUT FEATURES ###########################################
x = Concatenate(axis = -1)([word_embedding, pos_embedding, gtags_embedding, char_enc, syll_enc, concepts_input, local_cont])

################## BiLSTM MAX SEQUENCE WORD LENGTH 50 ######################################################
x = Bidirectional(LSTM(units=LSTM_UNITS, return_sequences=True,recurrent_dropout=0.4))(x)
x = Dropout(0.4)(x)

################## DENSE LAYER ############################################################################
x = Dense(DENSE_UNITS, activation='tanh')(x)

################## CRF LAYER ####################################################################################
crf = CRF(len(tag2idx), sparse_target = False)
loss = crf.loss_function
y_output = crf(x)

model = Model(inputs = [char_input, word_input, pos_input, gtag_input, sylls_input, concepts_input], outputs = y_output)
model.compile(optimizer = "adam", loss = loss, metrics = [crf.accuracy])
model.summary()



### Training the model

1. Epochs: 5
2. Batch size: 32

In [ ]:
NUM_EPOCHS = 5
checkpoint = ModelCheckpoint(sst_home + 'model_final/best_model.hdf5', monitor='loss', verbose=1, save_best_only=True, mode='auto', period=1)
history = model.fit([X_char_train, X_words_train, X_pos_train, X_tag_train, X_syll_train, X_concepts_train], y_train,
                    batch_size = 32,
                    epochs = NUM_EPOCHS,
                    callbacks=[checkpoint])

In [ ]:
model.load_weights(sst_home + 'model_3/best_model.hdf5')

## Predictions

### Annotate Predictions

The following code generates the annotation files according to the predictions made by the model.

In [ ]:
def annotate(y_pred, offset, file_name, path_to_save):
  i = 1
  has_begin = False
  text = ''
  file = path_to_save + file_name + '.ann'
  for pred_sent, offset_sent in zip(y_pred, offset):
    #fixed_sent = fixSeqTagged(pred_sent, offset_sent)
    for ann, info in zip(pred_sent, offset_sent):
      if ann[0] == SINGLE:
        if has_begin:
          ##print('ERROR: Se ha comenzado una entidad sin terminar la anterior')
          text = text + 'T{}\tMORFOLOGIA_NEOPLASIA {} {}\t{}\n'.format(i, start, end, entity.replace('\n', '#SALTO'))
          i = i + 1
          has_begin = False

        text = text + 'T{}\tMORFOLOGIA_NEOPLASIA {} {}\t{}\n'.format(i, info[1], info[2], info[0].replace('\n', '#SALTO'))
        i = i + 1

      if ann[0] == BEGIN:
        if has_begin:
          ##print('ERROR: Se ha comenzado una entidad sin terminar la anterior')
          text = text + 'T{}\tMORFOLOGIA_NEOPLASIA {} {}\t{}\n'.format(i, start, end, entity.replace('\n', '#SALTO'))
          i = i + 1

        entity = info[0]
        start = info[1]
        end = info[2]
        has_begin = True

      if ann[0] == INSIDE:
        if not has_begin:
          ##print('ERROR: Estamos en mitad de una entidad sin haber comenzado')
          entity = info[0]
          start = info[1]
          end = info[2]
          has_begin = True

        else:
          entity = entity + ' ' + info[0]
          end = info[2]
      
      if ann[0] == END:
        if not has_begin:
          ##print('ERROR: Alcanzamos el final de una entidad sin tener principio')
          text = text + 'T{}\tMORFOLOGIA_NEOPLASIA {} {}\t{}\n'.format(i, info[1], info[2], info[0].replace('\n', '#SALTO'))
          i = i + 1
        else:
          entity_text = entity + ' ' + info[0]
          text = text + 'T{}\tMORFOLOGIA_NEOPLASIA {} {}\t{}\n'.format(i, start, info[2], entity_text.replace('\n', '#SALTO'))
          has_begin = False
          i = i + 1

  with open(file, 'w') as f:
    f.write(text)

In [ ]:
import numpy as np

df_concepts = pd.read_csv(sst_home + 'info_test.csv')
df_concepts.drop_duplicates()
df_concepts = df_concepts[df_concepts['CLASE'].isin(classes)]
sst_home_test = sst_home + 'test-background-set-to-publish/'
_, _docs, _docs_tags, offset = getElements(sst_home_test, MAX_SEQ_LEN, df_concepts, classes, getTags = False)
idx2tag = {idx:tag for (tag, idx) in tag2idx.items()}

result_path = sst_home + 'final-result/'
for (file_name, doc_sents) in _docs.items():
  X_char = getCharacterInput(_docs[file_name][0], MAX_SEQ_LEN, MAX_CHARS_LEN, char2idx)
  X_words = getWordInput(_docs[file_name][0], MAX_SEQ_LEN, word2idx)
  X_pos = getPosInput(_docs[file_name][1], MAX_SEQ_LEN, pos2idx)
  X_tag = getGTagInput(_docs[file_name][2], MAX_SEQ_LEN, grammtags2idx)
  X_syll = getSyllsInput(_docs[file_name][0], MAX_SEQ_LEN, MAX_SYLLS_LEN, sylls2idx)
  X_concepts = getConceptsInput(_docs[file_name][3], MAX_SEQ_LEN, classes)

  #y_test = getY(_tags, MAX_SEQ_LEN, tag2idx)
  y_pred = model.predict([X_char, X_words, X_pos, X_tag, X_syll, X_concepts])
  y_pred = [list(map(lambda x: idx2tag[np.argmax(x)], sent)) for sent in y_pred]
  offset_test = offset[file_name]
  annotate(y_pred, offset_test, file_name, result_path)

In [ ]:
pred_data = [_docs, _docs_tags, offset, idx2tag]
with open(sst_home + 'data_final/prediction.pickle', 'wb') as handle:
    pickle.dump(pred_data, handle, protocol=pickle.HIGHEST_PROTOCOL)